661: Python Function to find the maximum sum that can be formed which has no three consecutive elements present.

In [54]:
%%writefile bugged_max_sum.py

def max_sum_of_three_consecutive(arr, n): 
	sum = [0 for k in range(n)] 
	if n >= 1: 
		sum[0] = arr[0] 
	if n >= 2: 
		sum[1] = arr[0] + arr[1] 
	if n > 2: 
		sum[2] = max(sum[1], max(arr[1] + arr[2], arr[0] + arr[2])) 
	for i in range(3, n): 
		sum[i] = max(max(sum[i-1], sum[i-2] + arr[i]), arr[i] + arr[i-1] + sum[i-4]) # Bug: used sum[i-4] instead of sum[i-3]
	return sum[n-1]

Overwriting bugged_max_sum.py


1. LLM Testing

In [55]:
%%writefile llm_test_max_sum.py
import pytest
from bugged_max_sum import max_sum_of_three_consecutive

def test_single_element():
    assert max_sum_of_three_consecutive([5], 1) == 5


def test_two_elements():
    assert max_sum_of_three_consecutive([4, 7], 2) == 11


def test_three_elements_basic():
    # Best selection: take first and last elements
    assert max_sum_of_three_consecutive([3, 2, 5], 3) == 8


def test_four_elements():
    # Optimal: take elements at indices 0, 1, and 3
    arr = [1, 2, 3, 4]
    assert max_sum_of_three_consecutive(arr, 4) == 7


def test_five_elements():
    # Optimal: skip the middle element to maximize sum
    arr = [1, 2, 3, 4, 5]
    assert max_sum_of_three_consecutive(arr, 5) == 12


def test_uniform_array_length_six():
    # With all elements equal, should select 4 out of 6
    arr = [1, 1, 1, 1, 1, 1]
    assert max_sum_of_three_consecutive(arr, 6) == 4


def test_typical_case():
    # Should select largest values while respecting constraint
    arr = [3000, 2000, 1000, 3, 10]
    assert max_sum_of_three_consecutive(arr, 5) == 5010


def test_increasing_sequence():
    # Selection pattern for consecutive increasing numbers
    arr = [1, 2, 3, 4, 5, 6, 7]
    assert max_sum_of_three_consecutive(arr, 7) == 19


def test_large_values_at_end():
    # Should prioritize larger values at the end
    arr = [1, 1, 1, 100, 100]
    assert max_sum_of_three_consecutive(arr, 5) == 202


def test_alternating_high_low():
    # Pattern with alternating high and low values
    arr = [10, 1, 10, 1, 10, 1, 10]
    assert max_sum_of_three_consecutive(arr, 7) == 40

Overwriting llm_test_max_sum.py


In [56]:
!pytest -v llm_test_max_sum.py

============================= test session starts =============================
platform win32 -- Python 3.13.2, pytest-8.4.2, pluggy-1.6.0 -- C:\Python313\python.exe
cachedir: .pytest_cache
hypothesis profile 'default'
rootdir: d:\IIIT HYD\SSD\Final project\HumanVsLLM\661
plugins: anyio-4.9.0, hypothesis-6.147.0
collecting ... collected 10 items

llm_test_max_sum.py::test_single_element PASSED                          [ 10%]
llm_test_max_sum.py::test_two_elements PASSED                            [ 20%]
llm_test_max_sum.py::test_three_elements_basic PASSED                    [ 30%]
llm_test_max_sum.py::test_four_elements PASSED                           [ 40%]
llm_test_max_sum.py::test_five_elements FAILED                           [ 50%]
llm_test_max_sum.py::test_uniform_array_length_six PASSED                [ 60%]
llm_test_max_sum.py::test_typical_case PASSED                            [ 70%]
llm_test_max_sum.py::test_increasing_sequence FAILED                     [ 80%]
llm_test_m

2. Human Testing

In [57]:
%%writefile human_test_max_sum.py

from hypothesis import given, strategies as st, settings, assume
from bugged_max_sum import max_sum_of_three_consecutive


@given(st.lists(st.integers(min_value=1, max_value=100), min_size=1, max_size=20))
@settings(max_examples=100)
def test_result_with_total(arr):
    n = len(arr)
    result = max_sum_of_three_consecutive(arr, n)
    total = sum(arr)
    assert result <= total


@given(st.lists(st.integers(min_value=1, max_value=100), min_size=1, max_size=20))
@settings(max_examples=100)
def test_alternating_sum(arr):
    n = len(arr)
    result = max_sum_of_three_consecutive(arr, n)
    alternating_sum = sum(arr[i] for i in range(0, n, 2))
    assert result >= alternating_sum


@given(st.lists(st.integers(min_value=1, max_value=50), min_size=1, max_size=15))
@settings(max_examples=100)
def test_scaling_property(arr):
    n = len(arr)
    result = max_sum_of_three_consecutive(arr, n)
    
    doubled_arr = [x * 2 for x in arr]
    doubled_result = max_sum_of_three_consecutive(doubled_arr, n)
    
    assert doubled_result == result * 2


@given(st.lists(st.integers(min_value=1, max_value=50), min_size=1, max_size=8))
@settings(max_examples=100)
def test_check_all_possibilities(arr):
    n = len(arr)
    result = max_sum_of_three_consecutive(arr, n)

    best = 0
    for i in range(1, 2**n):
        positions = []
        for j in range(n):
            if i & (1 << j):
                positions.append(j)

        has_three_consecutive = False
        if len(positions) >= 3:
            for k in range(len(positions) - 2):
                if positions[k+1] == positions[k] + 1 and positions[k+2] == positions[k] + 2:
                    has_three_consecutive = True
                    break
        
        if not has_three_consecutive:
            total = sum(arr[p] for p in positions)
            if total > best:
                best = total
    
    assert result == best

Overwriting human_test_max_sum.py


In [58]:
!pytest -v human_test_max_sum.py

============================= test session starts =============================
platform win32 -- Python 3.13.2, pytest-8.4.2, pluggy-1.6.0 -- C:\Python313\python.exe
cachedir: .pytest_cache
hypothesis profile 'default'
rootdir: d:\IIIT HYD\SSD\Final project\HumanVsLLM\661
plugins: anyio-4.9.0, hypothesis-6.147.0
collecting ... collected 4 items

human_test_max_sum.py::test_result_with_total PASSED                     [ 25%]
human_test_max_sum.py::test_alternating_sum PASSED                       [ 50%]
human_test_max_sum.py::test_scaling_property PASSED                      [ 75%]
human_test_max_sum.py::test_check_all_possibilities FAILED               [100%]

================================== FAILURES ===================================
________________________ test_check_all_possibilities _________________________

    @given(st.lists(st.integers(min_value=1, max_value=50), min_size=1, max_size=8))
>   @settings(max_examples=100)
                   ^^^

human_test_max_sum.py:37: 
_